In [1]:
%load_ext autoreload
%autoreload 2
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.opentrons_backend import OpentronsBackend 
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend
from pylabrobot.resources import Coordinate
from pylabrobot.resources.opentrons import OTDeck # pylabrobot.liquid_handling.resources is now pylabrobot.resources

from pylabrobot.resources.opentrons import (
    opentrons_96_tiprack_20ul    
)
from pylabrobot.resources.corning_costar.plates import  (
    Cos_96_Rd
)
import pandas as pd
import gradio as gr


c:\Users\JBagley\pyLabRobot source\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
supported_plates = {'Cos_96_Rd':Cos_96_Rd}
'''

directions = FileUpload(
        accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False,  # True to accept multiple files upload else False
        description='Upload csv file'
    )
    
Source = Dropdown(
    options=supported_plates.keys(),
    #value=options[1],
    description='Source plate type',
    disabled=False,
    )
Destination = Dropdown(
    options=supported_plates.keys(),
    #value=options[1],
    description='destination plate type',
    disabled=False,
    )
'''
#display(directions)
#display(Source)
#display(Destination)




"\n\ndirections = FileUpload(\n        accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'\n        multiple=False,  # True to accept multiple files upload else False\n        description='Upload csv file'\n    )\n    \nSource = Dropdown(\n    options=supported_plates.keys(),\n    #value=options[1],\n    description='Source plate type',\n    disabled=False,\n    )\nDestination = Dropdown(\n    options=supported_plates.keys(),\n    #value=options[1],\n    description='destination plate type',\n    disabled=False,\n    )\n"

In addition, import the {class}`~pylabrobot.liquid_handling.resources.opentrons.OTDeck`, which represents the deck of the Opentron.

In [3]:
def prep_deck(device, Source_plate_type, Destination_plate_type):
    tip_rack = opentrons_96_tiprack_20ul(name='tip_rack')
    source_plate = supported_plates[Source_plate_type](name='source_plate')
    destination_plate = supported_plates[Destination_plate_type](name='destination_plate')
    
    device.deck.assign_child_at_slot(tip_rack, slot=1)
    device.deck.assign_child_at_slot(source_plate, slot=2)
    device.deck.assign_child_at_slot(destination_plate, slot=3)



In [4]:
async def program(device, instructions):
        
    #Source_plate_type = Source.value
    #Destination_plate_type = Destination.value
    tip_rack: TipRack = device.get_resource('tip_rack')
    source_plate: Plate = device.get_resource('source_plate')
    destination_plate: Plate = device.get_resource('destination_plate')
    
    #source_plate = supported_plates[Source_plate_type](name='source_plate')
    #destination_plate = supported_plates[Destination_plate_type](name='destination_plate')

    pipetting_tracker = pd.DataFrame(columns=instructions.columns)
    for i, row in instructions.iterrows():
        #tip_source = get_next_tip()
        tip_source = 'A1'
        await device.pick_up_tips(tip_rack[tip_source])
        await device.aspirate(source_plate[row.source_well], vols=[row.volume], offsets_z=[1])
        await device.dispense(destination_plate[row.destination_well], vols=[row.volume],  offsets_z=[1])
        pipetting_tracker = pd.concat([pipetting_tracker,row])
        print(f'{row.volume} transferred from {row.source_well} to {row.destination_well}.')
        await device.discard_tips(tip_rack[tip_source])
        

Create a new liquid handler using `OpentronsBackend` as its backend.

In [5]:
from time import sleep
OT_backend = OpentronsBackend(host="169.254.137.37", port=31950) #169.254.121.96
simulator = SimulatorBackend() #169.254.121.96

async def run_protocol(source_plate_type, destination_plate_type, directions):
    directions = pd.read_csv(directions)
    #lh = LiquidHandler(backend=backend, deck=OTDeck())
    #sim = LiquidHandler(backend=simulator, deck=OTDeck())   
    lh = LiquidHandler(backend=OT_backend, deck=OTDeck())
    sim = LiquidHandler(backend=simulator, deck=OTDeck())

    await sim.setup()
    simulator.wait_for_connection() # wait for the browser to connect
    prep_deck(sim, source_plate_type, destination_plate_type)

    # disable PLR volume trakcing
    from pylabrobot.resources import set_volume_tracking
    set_volume_tracking(enabled=False)

    # update the simulator state with tips and volume
    #await simulator.fill_tip_rack(sim.deck.get_resource("tip_rack"))
    #await simulator.adjust_well_volume(sim.deck.get_resource("source_plate"), pattern=[[400]*12]*8)

    await program(sim, directions)
    sim.close()
    return None


In [6]:
with gr.Blocks() as demo:
    plate_options = list(supported_plates.keys())
    Source_plate_type = gr.Dropdown(choices=plate_options, label='Select source plate type', value='Cos_96_Rd')
    Destination_plate_type = gr.Dropdown(choices=plate_options, label='Select Destination plate type', value='Cos_96_Rd')
    instructions = gr.File(label='upload instructions file')
    continue_button = gr.Button("submit")
    continue_button.click(run_protocol, inputs = [Source_plate_type, Destination_plate_type, instructions])

#demo.launch(share=False, debug=True)

In [7]:
import asyncio
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.ensure_future(run_protocol('Cos_96_Rd', 'Cos_96_Rd', 'test_transfer.csv')))
#asyncio.run(run_protocol('Cos_96_Rd', 'Cos_96_Rd', 'test_transfer.csv'))

RuntimeError: This event loop is already running

Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.
Websocket server started at http://127.0.0.1:2122
File server started at http://127.0.0.1:1337 . Open this URL in your browser.
